In [343]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [344]:
dataset = {}

In [345]:
# loading data file

In [346]:
path = "C:\\Users\\KangYounKook\\Desktop\\ML_Study\\"

In [347]:
filename = "sectionInfo.txt"

In [348]:
def getDatasetFromFile(filename):
    dataset = {}
    with open(filename, 'r') as f:
        lines = f.readlines()
        
        for line in lines:
            words = line.strip('\n').split(',')
    
            sectionId = words[0]
            sectionType = words[1]
            distance = words[3]
            speed = words[4]
    
            incount = int(words[5])
            incomings = []
            for i in range(6, 6+incount):
                incomings.append(words[i])
        #    print(incomings)
            outcount = int(words[6+incount])
            outgoings = []
            for i in range(7+incount, 7+incount+outcount):
                outgoings.append(words[i])
        #    print(outgoings)
    
            value = [sectionType, distance, speed, incomings, outgoings]
            dataset[sectionId] = value
            #printDataset(sectionId)
        print('getDataFromFile ',filename)
    return dataset

In [349]:
def printDataset(sectionId):
    print(sectionId)
    print(dataset[sectionId])

In [350]:
dataset = getDatasetFromFile(path+filename)

getDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\sectionInfo.txt


In [351]:
def updateDatasetFromFile(filename):
    with open(filename,'r') as f:
        lines = f.readlines()
        for line in lines:
            words = line.strip('\n').split(',')
            sectionId = words[0]
            move_count = int(words[6])
            work_count = int(words[7])
            ratio = int(words[8])
            
            (dataset[sectionId]).append(move_count)
            (dataset[sectionId]).append(work_count)
            (dataset[sectionId]).append(ratio)
            #printDataset(sectionId)
    print('loadDataFromFile ', filename)
    print('update Dataset [move_count,work_count,ratio]')        

In [352]:
updateDatasetFromFile(path+"PeriodTotalAverage.txt")

loadDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\PeriodTotalAverage.txt
update Dataset [move_count,work_count,ratio]


In [353]:
#dataset

In [354]:
filename = "input10_all.csv"

In [355]:
def loadCsvInput(path, filename):
    fullname = path+filename
    print('loadCsvInput ',fullname)
    return pd.read_csv(fullname)

In [356]:
data = loadCsvInput(path,filename)

loadCsvInput  C:\Users\KangYounKook\Desktop\ML_Study\input10_all.csv


In [357]:
# load sectionInfo
# set section data
# load inputdata
# build modelInput

In [358]:
dataAll = np.array(data)
print(dataAll.shape)

(1440, 146)


In [359]:
def getPrevId(id):
    value = dataset[id]
    maxcount = -1
    maxid = id
    for prev in value[3]:
#        print(prev)
        if (dataset[prev] is None):
            print('dataset[prev] is None')
            continue
#        print(dataset[prev][5])
#        print(data[prev])
        if len(dataset[prev]) < 8:
            continue
        if (maxcount < dataset[prev][5]):
            maxcount = dataset[prev][5]
            maxid = prev

    if maxcount == -1:
        return None
            
    return maxid

In [360]:
def getNextId(id):
    value = dataset[id]
    maxcount = -1
    maxid = id
    for nxt in value[4]:
#        print(nxt)
        if (dataset[nxt] is None):
            print('dataset[prev] is None')
            continue
#        print(dataset[nxt][5])
#        print(data[nxt])
        if len(dataset[nxt]) < 8:
            continue
        if (maxcount < dataset[nxt][5]):
            maxcount = dataset[nxt][5]
            maxid = nxt
            
    if maxcount == -1:
        return None
    return maxid

In [361]:
#    if id not in data.index:

In [362]:
def getX(id):
    if id not in data.columns:
        return None
    curS = data[id]
    prevId = getPrevId(id)
    if prevId not in data.columns:
        print(prevId, 'data[prevId] is None')
        return None
    prevS = data[prevId]
    prevId2 = getPrevId(prevId)
    if prevId2 not in data.columns:
        print(prevId2, 'data[prevId] is None')
        return None
    prevS2 = data[prevId2]    
    nextId = getNextId(id)
    if nextId not in data.columns:
        print(nextId, 'data[prevId] is None')
        return None
    nextS = data[nextId]
    nextId2 = getNextId(nextId)
    if nextId2 not in data.columns:
        print(nextId2, 'data[prevId] is None')
        return None
    nextS2 = data[nextId2]
    X = np.array(np.c_[prevS2, prevS, curS, nextS, nextS2])
    print(prevId,prevId2,id,nextId,nextId2)
#    print(X.shape)
    return X

In [363]:
#print(data.columns)
#if 'Section676' in data.columns:
#if 'Section676d' not in data.columns:
print(dataset['Section2262'])
print(dataset['Section4215'])
if 'Section4215' not in data.columns:
    print('no')
print(len(dataset['Section2262']))
print(len(dataset['Section4215']))


['D_C', '2598', '700', ['Section2261'], ['Section4215'], 2682, 0, 89]
['C_D', '5460', '3309', ['Section3625', 'Section2262'], ['Section3445', 'Section378']]
no
8
5


In [342]:
num = 0
#selectedList = ['Section697','Section754','Section1212','Section852']
for id in dataset.keys():
#for id in selectedList:

    if id not in data.columns:
        continue
    if dataset[id][5] < 1440*2:
        continue
    num += 1
    print(num, id, '-------------------------')
    print(dataset[id])
    
    Xa = getX(id)
    if Xa is None:
        print('X is none')
        continue
    Ya = np.array(data[id]).reshape(-1,1)
#    print(Ya.shape)
    Xt = Xa[0:1000]
    Xval = Xa[1000:1400]
    
    Yt = Ya[1:1001]
    Yval = Ya[1001:1401]
    
    input_size = 5
    classes = 5
    X = tf.placeholder(tf.float32, [None, input_size])
    Y = tf.placeholder(tf.int32, [None, 1])
    Y_one_hot = tf.one_hot(Y, classes)
    Y_one_hot = tf.reshape(Y_one_hot, [-1, classes])

    W = tf.Variable(tf.random_normal([input_size, classes]), name='weight')
    b = tf.Variable(tf.random_normal([classes]), name='bias')

    logits = tf.matmul(X, W) + b
    hypothesis = tf.nn.softmax(logits)

    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
    cost = tf.reduce_mean(cost_i)

    optimizer = tf.train.AdamOptimizer().minimize(cost)
    prediction = tf.argmax(hypothesis, 1)
    correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for step in range(2000):
            sess.run(optimizer, feed_dict={X:Xt, Y:Yt})
#            if step % 2000 == 0:
#                loss, acc = sess.run([cost,accuracy], feed_dict={X:Xt, Y:Yt})
#                print("Step:{:5} Loss:{:.3f} Acc:{:.2%}".format(step, loss, acc))
        print('accuracy:', sess.run(accuracy, feed_dict={X:Xt,Y:Yt}))
        print('accuracy:', sess.run(accuracy, feed_dict={X:Xval,Y:Yval}))
    

1 Section760 -------------------------
['D_C', '2503', '3302', ['Section540'], ['Section537'], 4588, 0, 41]
Section540 Section776 Section760 Section537 Section3840
accuracy: 0.316
accuracy: 0.335
2 Section1212 -------------------------
['C_D', '6943', '3310', ['Section740', 'Section3806'], ['Section3787', 'Section732'], 4408, 0, 62]
Section3806 Section3807 Section1212 Section732 Section1176
accuracy: 0.791
accuracy: 0.785
3 Section700 -------------------------
['D_D', '2654', '3305', ['Section3098'], ['Section789', 'Section4142'], 3767, 0, 25]
Section3098 Section4334 Section700 Section789 Section4634
accuracy: 0.439
accuracy: 0.4275
4 Section776 -------------------------
['D_C', '11810', '3309', ['Section583'], ['Section540'], 3795, 0, 63]
Section583 Section4133 Section776 Section540 Section760
accuracy: 0.442
accuracy: 0.3875
5 Section732 -------------------------
['D_C', '5516', '3304', ['Section1212'], ['Section1176'], 3143, 0, 61]
Section1212 Section3806 Section732 Section1176 Sect

accuracy: 0.713
accuracy: 0.665
42 Section3757 -------------------------
['D_C', '2310', '400', ['Section1250'], ['Section975'], 2938, 0, 77]
Section1250 Section4049 Section3757 Section975 Section4133
accuracy: 0.711
accuracy: 0.7125
43 Section4463 -------------------------
['C_D', '2238', '3305', ['Section695', 'Section697'], ['Section785', 'Section4139'], 5256, 52, 28]
None data[prevId] is None
X is none
44 Section3847 -------------------------
['D_C', '1976', '700', ['Section1132'], ['Section537'], 3086, 0, 89]
Section1132 Section719 Section3847 Section537 Section3840
accuracy: 0.906
accuracy: 0.8725
45 Section496 -------------------------
['C_D', '1939', '3308', ['Section758', 'Section3838'], ['Section754', 'Section3836'], 3127, 0, 27]
Section758 Section537 Section496 Section3836 Section1172
accuracy: 0.811
accuracy: 0.84
46 Section1281 -------------------------
['C_D', '777', '1000', ['Section3985', 'Section1842'], ['Section718', 'Section3778'], 4489, 21, 30]
Section1842 Section12

In [269]:
# data preprocessing for input

In [270]:
def model(input_size, classes):
    X = tf.placeholder(tf.float32, [None, input_size])
    Y = tf.placeholder(tf.int32, [None, 1])
    Y_one_hot = tf.one_hot(Y, classes)
    Y_one_hot = tf.reshape(Y_one_hot, [-1, classes])

    W = tf.Variable(tf.random_normal([input_size, classes]), name='weight')
    b = tf.Variable(tf.random_normal([classes]), name='bias')

    logits = tf.matmul(X, W) + b
    hypothesis = tf.nn.softmax(logits)

    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
    cost = tf.reduce_mean(cost_i)
    return hypothesis, cost

In [ ]:
#tf.contrib.lookup.index_table_from_tensor